# 错误与异常
首先要了解，Python中的错误和异常是什么？两者之间又有什么联系和区别呢？
通常来说，程序中的错误至少包括两种，一种是语法错误，另一种则是异常。
所谓语法错误，你应该很清楚，也就是你写的代码不符合编程规范，无法被识别与执行，比如下面这个例
子：

In [1]:
if name is not None
print(name)

SyntaxError: invalid syntax (<ipython-input-1-46d977065f07>, line 1)

If语句漏掉了冒号，不符合Python的语法规范，所以程序就会报错invalid syntax。
而异常则是指程序的语法正确，也可以被执行，但在执行过程中遇到了错误，抛出了异常，比如下面的3个
例子：

Traceback (most recent call last):
File "<stdin>", line 1, in <module>
ZeroDivisionError: integer division or modulo by zero
    
order * 2
Traceback (most recent call last):
File "<stdin>", line 1, in <module>
NameError: name 'order' is not defined
    
1 + [1, 2]
Traceback (most recent call last):
File "<stdin>", line 1, in <module>
TypeError: unsupported operand type(s) for +: 'int' and 'list'
型和一个列表相加也是不可取的。
于是，当程序运行到这些地方时，就抛出了异常，并且终止运行。例子中的ZeroDivisionError
NameError和TypeError，就是三种常见的异常类型。
当然，Python中还有很多其他异常类型，比如KeyError是指字典中的键找不到；FileNotFoundError
是指发送了读取文件的请求，但相应的文件不存在等等，我在此不一一赘述，你可以自行参考相应文档。

## 如何处理异常
刚刚讲到，如果执行到程序中某处抛出了异常，程序就会被终止并退出。你可能会问，那有没有什么办法可
以不终止程序，让其照样运行下去呢？答案当然是肯定的，这也就是我们所说的异常处理，通常使用try和
except来解决，比如：

In [4]:
try:
    s = input('please enter two numbers separated by comma: ')
    num1 = int(s.split(',')[0].strip())
    num2 = int(s.split(',')[1].strip())

except ValueError as err:
    print('Value Error: {}'.format(err))
    print('continue')

please enter two numbers separated by comma: a
Value Error: invalid literal for int() with base 10: 'a'
continue


这里默认用户输入以逗号相隔的两个整形数字，将其提取后，做后续的操作（注意input函数会将输入转换
为字符串类型）。如果我们输入a,b，程序便会抛出异常invalid literal for int() with base
10: 'a'，然后跳出try这个block。
由于程序抛出的异常类型是ValueError，和except block所catch的异常类型相匹配，所以except block便会
被执行，最终输出Value Error: invalid literal for int() with base 10: 'a'，并打印出
continue。

我们知道，except block只接受与它相匹配的异常类型并执行，如果程序抛出的异常并不匹配，那么程序照
样会终止并退出。
所以，还是刚刚这个例子，如果我们只输入1，程序抛出的异常就是IndexError: list index out of
range，与ValueError不匹配，那么except block就不会被执行，程序便会终止并退出（continue不会被打
印）。

In [5]:
try:
    s = input('please enter two numbers separated by comma: ')
    num1 = int(s.split(',')[0].strip())
    num2 = int(s.split(',')[1].strip())

except ValueError as err:
    print('Value Error: {}'.format(err))
    print('continue')

please enter two numbers separated by comma: 1


IndexError: list index out of range

不过，很多时候，我们很难保证程序覆盖所有的异常类型，所以，更通常的做法，是在最后一个except
block，声明其处理的异常类型是Exception。Exception是其他所有非系统异常的基类，能够匹配任意非系
统异常。那么这段代码就可以写成下面这样：

In [6]:
try:
    s = input('please enter two numbers separated by comma: ')
    num1 = int(s.split(',')[0].strip())
    num2 = int(s.split(',')[1].strip())

except ValueError as err:
    print('Value Error: {}'.format(err))
except IndexError as err:as
    
    print('Index Error: {}'.format(err))
except Exception as err:
    print('Other error: {}'.format(err))
    print('continue')

please enter two numbers separated by comma: a
Value Error: invalid literal for int() with base 10: 'a'


需要注意，当程序中存在多个except block时，最多只有一个except block会被执行。换句话说，如果多个
except声明的异常类型都与实际相匹配，那么只有最前面的except block会被执行，其他则被忽略。
异常处理中，还有一个很常见的用法是finally，经常和try、except放在一起来用。无论发生什么情况，
finally block中的语句都会被执行，哪怕前面的try和excep block中使用了return语句。
一个常见的应用场景，便是文件的读取：

In [11]:
import sys
try:
    f=open('in.txt', 'r')
    f.read()
except OSError as err:
    print('OS error: {}'.format(err))
except:
    print('Unexpected error:', sys.exc_info()[0])
finally:
    f.close()

这段代码中，try block尝试读取file.txt这个文件，并对其中的数据进行一系列的处理，到最后，无论是读取
成功还是读取失败，程序都会执行finally中的语句——关闭这个文件流，确保文件的完整性。因此，在
finally中，我们通常会放一些无无论论如如何何都都要要执执行行的语句。
值得一提的是，对于文件的读取，我们也常常使用with open，你也许在前面的例子中已经看到过，with
open会在最后自动关闭文件，让语句更加简洁。

## 用户自定义异常
前面的例子里充斥了很多Python内置的异常类型，你可能会问，我可以创建自己的异常类型吗？
答案是肯定是，Python当然允许我们这么做。下面这个例子，我们创建了自定义的异常类型
MyInputError，定义并实现了初始化函数和str函数（直接print时调用）：

In [16]:
class MyInputError(Exception):
# """Exception raised when there're errors in input"""
    def __init__(self, value): # ⾃定义异常类型的初始化
        self.value = value
    def __str__(self): # ⾃定义异常类型的string表达形式
        return ("{} is invalid input".format(repr(self.value)))
try:
    raise MyInputError(1) # 抛出MyInputError这个异常
except MyInputError as err:
    print('error: {}'.format(err))

error: 1 is invalid input


异异常常的的使使用用场场景景与与注注意意点点
学完了前面的基础知识，接下来我们着重谈一下，异常的使用场景与注意点。
通常来说，在程序中，**如果我们不确定某段代码能否成功执行，往往这个地方就需要使用异常处理。**除了上
述文件读取的例子，我可以再举一个例子来说明。
大型社交网站的后台，需要针对用户发送的请求返回相应记录。用户记录往往储存在key-value结构的数据
库中，每次有请求过来后，我们拿到用户的ID，并用ID查询数据库中此人的记录，就能返回相应的结果。
而数据库返回的原始数据，往往是json string的形式，这就需要我们首先对json string进行decode（解
码），你可能很容易想到下面的方法：

In [17]:
import json
raw_data = queryDB(uid) # 根据⽤⼾的id，返回相应的信息
data = json.loads(raw_data)

NameError: name 'queryDB' is not defined

这样的代码是不是就足够了呢？
要知道，在json.loads()函数中，输入的字符串如果不符合其规范，那么便无法解码，就会抛出异常，因此
加上异常处理十分必要。

In [ ]:
try:
    data = json.loads(raw_data)

except JSONDecodeError as err:
    print('JSONDecodeError: {}'.format(err))

不过，有一点切记，我们不能走向另一个极端——滥用异常处理。
比如，当你想要查找字典中某个键对应的值时，绝不能写成下面这种形式：

In [19]:
d = {'name': 'jason', 'age': 20}
try:
    value = d['dob']

except KeyError as err:
    print('KeyError: {}'.format(err))

KeyError: 'dob'


诚然，这样的代码并没有bug，但是让人看了摸不着头脑，也显得很冗余。如果你的代码中充斥着这种写
法，无疑对阅读、协作来说都是障碍。因此，对于**flow-control（流程控制）**的代码逻辑，我们一般不用异
常处理。
字典这个例子，写成下面这样就很好。



In [24]:
# 这里是判断键是否在
if 'dob' in d:
#     value = d['dob']
    print('yes')
else:
    print('no')

yes


# 总结
这节课， 我们一起学习了Python的异常处理及其使用场景，你需要重点掌握下面几点。

异常，通常是指程序运行的过程中遇到了错误，终止并退出。我们通常使用try except语句去处理异常，
这样程序就不会被终止，仍能继续执行。
处理异常时，如果有必须执行的语句，比如文件打开后必须关闭等等，则可以放在finally block中。
异常处理，通常用在你不确定某段代码能否成功执行，也无法轻易判断的情况下，比如数据库的连接、读
取等等。正常的flow-control逻辑，不要使用异常处理，直接用条件语句解决就可以了。

思考题
最后，给你留一个思考题。在异常处理时，如果try block中有多处抛出异常，需要我们使用多个try except
block吗？以数据库的连接、读取为例，下面两种写法，你觉得哪种更好呢？
>第一种：
try:
db = DB.connect('<db path>') # 可能会抛出异常
raw_data = DB.queryData('<viewer_id>') # 可能会抛出异常
except (DBConnectionError, DBQueryDataError) err:
print('Error: {}'.format(err))
    
>第二种：
try:
db = DB.connect('<db path>') # 可能会抛出异常
try:
raw_data = DB.queryData('<viewer_id>')
except DBQueryDataError as err:
print('DB query data error: {}'.format(err))
except DBConnectionError as err:
print('DB connection error: {}'.format(err))
    
    欢迎在留言区写下你的答案，还有你今天学习的心得和疑惑，也欢迎你把这篇文章分享给你的同事、朋友。
处理异常时，如果有必须执行的语句，比如文件打开后必须关闭等等，则可以放在finally block中。
异常处理，通常用在你不确定某段代码能否成功执行，也无法轻易判断的情况下，比如数据库的连接、读
取等等。正常的flow-control逻辑，不要使用异常处理，直接用条件语句解决就可以了。

第一种写法更加简洁，易于阅读。而且except后面的错误类型先抛出数据库连接错误，之后才抛出查询错
误，实现的异常处理和第二种一样。